In [2]:
!pip install pkuseg

In [37]:
import pkuseg
import os 
from functools import reduce
seg = pkuseg.pkuseg()           # 加载web模型
stop_dir='./stopwords/' #链接: https://pan.baidu.com/s/1lkpBp8JDTN14-Gnczi6_nA  密码: u21o--来自百度网盘超级会员V3的分享


#加载停用词
def read_f(file):
    if '.txt' not in file: 
        return []
    with open(file) as f: 
        stop_l=[i.strip('\n') for i in f.readlines()]
    return stop_l

stop_l=reduce(lambda x,y: x+y,map(read_f,[stop_dir+t for t in os.listdir(stop_dir)]))
stop_l+=["['","']"]

In [2]:
def my_cut(sentence):
    return [i for i in seg.cut(sentence) if i not in stop_l]

In [3]:
import pandas as pd
from pathlib import Path

In [4]:
DATA_STORE=Path('data.h5') 

## 存入数据

In [5]:
idx = pd.IndexSlice
num=1
def get_by_index(num,only_all=False):
    if num<10:
        num='url%d'%num
    else:
        num=str(num)
    with pd.HDFStore('./data.h5') as store:
        if only_all:
            all_df = (store['course/text/all/all_%s'%num])
            return all_df 
        else:
            all_df = (store['course/text/all/all_%s'%num])
            time_df= (store['course/text/detail/detail_%s'%num])
            return all_df,time_df 

In [6]:
vid_url=pd.read_csv('./url_.csv',header=None)

### 存入url数据

In [8]:
with pd.HDFStore(DATA_STORE) as store: 
    store.put('course/video/url',vid_url)

In [8]:
with pd.HDFStore(DATA_STORE) as store:
    url_=store['course/video/url']

In [9]:
url_.iloc[1].values[0]

'https://vods.pku.edu.cn/vod/httpDownload/2021-05-10/379200F9481D92DBD429C1F9C5338565.mp4'

In [14]:
import os 

In [27]:
wb_all_path,wb_time_path='./文本数据/all/','./文本数据/detail/'
all_list=os.listdir(wb_all_path)
time_list=os.listdir(wb_time_path)
all_list.sort()
time_list.sort()

with pd.HDFStore(DATA_STORE) as store:
    for i in range(len(all_list)):
        time_df=pd.read_csv(wb_time_path+time_list[i])
        all_df=pd.read_csv(wb_all_path+all_list[i])
        store.put('course/text/all/%s'%all_list[i][:-4],all_df)
        store.put('course/text/detail/%s'%time_list[i][:-4],time_df)
    store.put('course/text/len',pd.Series(len(all_list),name='length'))

In [29]:
with pd.HDFStore(DATA_STORE) as store:
    tmep=store['course/text/len']

### 倒排索引

In [4]:
with pd.HDFStore(DATA_STORE) as store:
    url_=store['course/video/url']

In [6]:
import re 

In [43]:
tokens=my_cut(all_df.result.values[0])

In [54]:
daopai_l=[]
counter=0
num=0
with pd.HDFStore('./data.h5') as store:
    while counter<store['course/text/len'].values[0]:
        try:
            all_df=get_by_index(num,only_all=True)
            print(all_df.id.values[0])
            counter+=1
            num+=1
            tokens=my_cut(all_df.result.values[0])
            daopai_l.append((tokens,all_df.id.values[0]))
        except KeyError:
            num+=1
        

url0
url1
url2
url3
url4
url5
url6
url7
url8
url9
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
30
31
32
33
34
35
36
38
41
42
43
44
45
46
47


In [73]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('course/text/daopai',pd.DataFrame(daopai_l,columns=['tokens','url_idx'],dtype=str))

In [34]:
with pd.HDFStore(DATA_STORE) as store:
    dd=store['course/text/daopai']

### 描述性

In [37]:
meta={}
for i in dd.url_idx.values:
    if 'url' in i and i[-1]<'8':#0-7
        meta[i]=['管理信息系统','管理/信息系统','陈文广','2021春倒数第%s节课'%i[-1]]
    elif 'url' in i and i[-1]>='8':
        meta[i]=['金融计量','金融/计量/时间序列','胡博','2021春倒数第%s'%(int(i[-1])-7)]
    elif i in [str(j) for j in range(10,24)] :
        meta[i]=['金融计量','金融/计量/时间序列','胡博','2021春倒数第%s'%(int(i)-9)]
    elif i in [str(j) for j in range(24,35)]:
        meta[i]=['Python数据分析','Python/数据分析','步一/孟凡','2021春倒数第%s'%(int(i)-23)]
    elif i in [str(j) for j in range(35,49)]:
        meta[i]=['信息存储与检索','信存检','赵丹群','2021春倒数第%s'%(int(i)-34)]

In [39]:
meta_df=pd.DataFrame(meta).T
meta_df.columns=['name','kw','teacher','time']

In [40]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('course/meta',meta_df)

In [9]:
num='url8'
with pd.HDFStore(DATA_STORE) as store:
    time_df=store['course/text/detail/detail_%s'%num]

In [41]:
with pd.HDFStore(DATA_STORE) as store:
    url_=store['course/video/url']
    dp=store['course/text/daopai']  
    meta=store['course/meta']  

### TF-IDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
contain_=['url%s'%i for i in vid_url.index if i<10 ]+['%s'%i for i in vid_url.index if i>=10 ]

In [71]:
all_df=pd.DataFrame()
contain_true=[]
with pd.HDFStore(DATA_STORE) as store:
    for i in contain_:
        try:
            all_df=all_df.append(store['course/text/all/all_%s'%i])
            contain_true.append(i)
        except KeyError:
            continue

In [29]:
with pd.HDFStore(DATA_STORE) as store:
    dp=store['course/text/daopai']

In [59]:
tv = TfidfVectorizer(use_idf=True, smooth_idf=True, norm=None)

In [52]:
corpus=dp.tokens.apply(lambda x:' '.join(eval(x))).tolist()

In [60]:
tf_idf=tv.fit(corpus)

In [62]:
corpus_vec=tf_idf.transform(corpus)

In [74]:
tf_idf_df=pd.DataFrame(corpus_vec.toarray(),columns=tf_idf.get_feature_names(),index=contain_true)

In [76]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('course/tfidf',tf_idf_df)

In [112]:
#after cut
def query_sort(input_):
    tokens=my_cut(input_)
    res_sr=reduce(lambda x,y:x+y,map(lambda x:tf_idf_df[x] if x in tf_idf_df.columns else 0,tokens))
    try:
        state=True
        i=3
        while state:
            if res_sr.sort_values(ascending=False).iloc[i-1]!=0:
                state=False
                return res_sr.sort_values(ascending=False).iloc[:i].index.tolist()#url_index
            else:
                i-=1
    except KeyError:#不在其中
        return '未找到关键词'

### TF-IDF 文档向量

In [115]:
tf_idf_corr=tf_idf_df.T.corr()

In [116]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('course/tfidf_corr',tf_idf_corr)

### word2vec 词向量

In [ ]:
!pip install gensim

In [12]:
from gensim.models import Word2Vec,word2vec
from functools import reduce

In [15]:
contain_=['url%s'%i for i in vid_url.index if i<10 ]+['%s'%i for i in vid_url.index if i>=10 ]

In [50]:
def detail_to_corpus(detail_df):
    return ' '.join(reduce(lambda x,y:x+['\n']+y,detail_df.res.apply(my_cut)))

In [51]:
contain_true=[]
with pd.HDFStore(DATA_STORE) as store:
    for i in contain_:
        try:
            detail_df=store['course/text/detail/detail_%s'%i]
            contain_true.append(i)
            with open('./word2vec/corpus/%s.txt'%i,'w+') as f:
                f.write(detail_to_corpus(detail_df))
        except KeyError:
            continue

In [54]:
from tqdm import tqdm

In [89]:
corpus_path='./word2vec/corpus/'
text_list=os.listdir(corpus_path)
for t in tqdm(text_list):
    if '.txt' in t:
        sentences=word2vec.Text8Corpus(corpus_path+t)
        w2v_model=Word2Vec(sentences=sentences,min_count=5,window=5,vector_size=50)
        w2v_model.save('./word2vec/model/%s.model'%t[:-4])

100%|██████████| 43/43 [00:01<00:00, 33.57it/s]


In [90]:
wc=WordCloud().add('相关词云',anl)
wc.render('./static/temp_wordcloud.html')

'/Users/fangbowen/Desktop/学校文件夹/21春/信息检索/v3/v3/test/static/temp_wordcloud.html'